# <center> 2차 크롤링 - <점핏> </center>

##### 링크들 들어가서 각각 정보 수집, 자율주행

* 기업이름
* 고용형태 : 신입, 경력
* 연봉 (대체로 면접후 결정 일겁니다, 크롤링 사이트 내용 그대로 검출하면 될듯!)
* 기업주소 : 기업 메인 위치
* 키워드 : 자율주행 / 드론 / 딥러닝 / 로봇 제어</br></br>
 -----뭉쳐서 '주요사항' column-----
* 자격 요건 : ROS, Python, C++, 학력
* 담당업무
* 우대사항

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"), options=options)

driver.get("https://www.jumpit.co.kr/search?locationTag=1&sort=relation&keyword=%EB%93%9C%EB%A1%A0")
time.sleep(2)

# 페이지 끝까지 스크롤
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

html_source = driver.page_source
soup = BeautifulSoup(html_source, "html.parser")
time.sleep(3)

job_box_1 = soup.find('section', class_='sc-c12e57e5-3 gjgpzi')
job_box_2 = job_box_1.find('section', class_='sc-80cff0b9-0 cOzTle')
job_box_3 = job_box_2.find_all('div', class_='sc-c8169e0e-0 gOSXGP')

image_link_list = []
category_list = []
major_work_list = []

for info in job_box_3:

    job_link = info.find('a')['href']
    image_link_list.append(job_link)

    major_work = info.find('h2', class_='position_card_info_title').text
    major_work_list.append(major_work)

    category_list.append('드론')

print(image_link_list, category_list)


In [2]:
print(len(image_link_list))
print(len(category_list))

10
10


In [3]:
import pandas as pd

df = pd.DataFrame({
    '키워드': category_list,
    'url': image_link_list,
    '담당업무' : major_work_list,
})

In [4]:
df.tail()

,키워드,url
5,드론,/position/20246
6,드론,/position/22255
7,드론,/position/21706
8,드론,/position/17024
9,드론,/position/13921


In [12]:
df.to_csv("../data/jumpit_drone_base.csv", sep =",", encoding = 'utf-8')

In [10]:
import pandas as pd

df = pd.read_csv("../data/jumpit_drone_base.csv", encoding='utf-8', index_col=0)

df['url'] = df['url'].apply(lambda x: f"https://www.jumpit.co.kr{x}")


In [7]:
import pandas as pd
df= pd.read_csv("../data/jumpit_drone_base.csv", encoding='utf-8', index_col=0)

In [13]:
df.tail()

,키워드,url
5,드론,https://www.jumpit.co.kr/position/20246
6,드론,https://www.jumpit.co.kr/position/22255
7,드론,https://www.jumpit.co.kr/position/21706
8,드론,https://www.jumpit.co.kr/position/17024
9,드론,https://www.jumpit.co.kr/position/13921


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import requests
import os
from tqdm.notebook import tqdm


options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={generate_user_agent()}") 

driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"),options=options)


company_address_list = []
company_name_list=[]
hire_list = []
major_work_list =[]
tech_list=[]
salary_list =[]


for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    driver.get(row['url'] )
    major_work_list_tmp = []

    time.sleep(1)  

    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, "html.parser")

    name_box = soup.find('section', class_='sc-3a5c981a-0 iORPxB')
    company_name = name_box.find('a').text
    company_name_list.append(company_name)

    company_address_box = soup.find('section', class_='sc-2da322c6-0 hWFbtF')
    company_address_box2 = company_address_box.find_all('dl', class_='sc-2da322c6-1 ksYhFT')

    company_address_list.append(company_address_box2[3].text)
    hire_list.append(company_address_box2[0].text)

    essential_tech_box = soup.find_all('dl', class_='sc-ae955bd-0 MHqtB')

    salary_list.append('회사내규에 따름')

    tech_stack = essential_tech_box[0].find('pre').text
    major_work_list_tmp.append(tech_stack)

    essential_tech1 = essential_tech_box[1].find('pre').text
    major_work_list_tmp.append(essential_tech1)

    essential_tech1 = essential_tech_box[2].find('pre').text
    major_work_list_tmp.append(essential_tech1)


    essential_tech2 = essential_tech_box[3].find('pre').text
    major_work_list_tmp.append(essential_tech2)
    
    
    tech_list.append(major_work_list_tmp)



    
    

In [15]:
print(len(company_name_list))
print(len(hire_list))
print(len(tech_list))
print(len(major_work_list))
print(len(company_address_list))

10
10
10
10
10


In [16]:
import pandas as pd

# 새로운 DataFrame 생성
new_data = pd.DataFrame({
    
    '기업이름' : company_name_list,
    '고용형태': hire_list,
    '연봉' : salary_list,
    '기업주소': company_address_list,
    '필요역량':tech_list
})

# 기존의 CSV 파일 불러오기
job_data = pd.read_csv('../data/jumpit_drone_base.csv')

# 두 DataFrame 병합
merged_data = pd.concat([job_data, new_data], axis=1)


# 결과 저장
merged_data.to_csv('../data/jumpit_drone_base2.csv', index=False)


In [17]:
merged_data.head()


,Unnamed: 0,키워드,url,기업이름,고용형태,연봉,기업주소,담당업무,필요역량
0,0,드론,https://www.jumpit.co.kr/position/19323,니어스랩,경력경력 2~10년,회사내규에 따름,"근무지역서울 강남구 논현로 417, 4,5층 4,5층지도보기·주소복사",• 드론으로 획득한 데이터를 관리하고 가공하는 B2B 웹 서비스 Front-end 개발,[JavaScriptTypeScriptReactReact Hook FormNext....
1,1,드론,https://www.jumpit.co.kr/position/17503,니어스랩,경력경력 3~10년,회사내규에 따름,"근무지역서울 강남구 논현로 417, 4,5층 4,5층지도보기·주소복사",• Arm 기반 Embedded Linux 개발\n- 시스템 구성에 필요한 부품 및...,"[C++ARMROSFWMCUEmbedded LinuxEmbeddedC, • Came..."
2,2,드론,https://www.jumpit.co.kr/position/8951,에너자이,경력신입~10년,회사내규에 따름,근무지역서울 강남구 테헤란로27길 27지도보기·주소복사,• Computer Vision 알고리즘 개발\n - 다양한 형태의 데이터 처리 ...,[AI/인공지능MachineLearningDeepLearningTensorFlowP...
3,3,드론,https://www.jumpit.co.kr/position/8955,에너자이,경력신입~10년,회사내규에 따름,근무지역서울 강남구 테헤란로27길 27지도보기·주소복사,• Edge device targeted AI compiler 개발\n• 경량화 파...,[C++HaskellDeepLearningF#TensorFlowScalaPyTorc...
4,4,드론,https://www.jumpit.co.kr/position/17677,에너자이,경력경력 2~3년,회사내규에 따름,근무지역서울 강남구 테헤란로27길 27지도보기·주소복사,• CCTV 카메라 파이프라인상에서 에너자이의 AI 모델이 원활하게 작동하도록 해요...,"[AI/인공지능C++Embedded LinuxEmbeddedC, • 임베디드, 모바..."


In [18]:
# 컬럼 재정렬 및 불필요한 컬럼 제거
merged_data = merged_data[['기업이름', '고용형태', '연봉', '기업주소', '키워드', '담당업무','필요역량']]

merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,니어스랩,경력경력 2~10년,회사내규에 따름,"근무지역서울 강남구 논현로 417, 4,5층 4,5층지도보기·주소복사",드론,• 드론으로 획득한 데이터를 관리하고 가공하는 B2B 웹 서비스 Front-end 개발,[JavaScriptTypeScriptReactReact Hook FormNext....
1,니어스랩,경력경력 3~10년,회사내규에 따름,"근무지역서울 강남구 논현로 417, 4,5층 4,5층지도보기·주소복사",드론,• Arm 기반 Embedded Linux 개발\n- 시스템 구성에 필요한 부품 및...,"[C++ARMROSFWMCUEmbedded LinuxEmbeddedC, • Came..."
2,에너자이,경력신입~10년,회사내규에 따름,근무지역서울 강남구 테헤란로27길 27지도보기·주소복사,드론,• Computer Vision 알고리즘 개발\n - 다양한 형태의 데이터 처리 ...,[AI/인공지능MachineLearningDeepLearningTensorFlowP...
3,에너자이,경력신입~10년,회사내규에 따름,근무지역서울 강남구 테헤란로27길 27지도보기·주소복사,드론,• Edge device targeted AI compiler 개발\n• 경량화 파...,[C++HaskellDeepLearningF#TensorFlowScalaPyTorc...
4,에너자이,경력경력 2~3년,회사내규에 따름,근무지역서울 강남구 테헤란로27길 27지도보기·주소복사,드론,• CCTV 카메라 파이프라인상에서 에너자이의 AI 모델이 원활하게 작동하도록 해요...,"[AI/인공지능C++Embedded LinuxEmbeddedC, • 임베디드, 모바..."


In [19]:
# 결과 저장
merged_data.to_csv('../data/project_eda_dataset/jumpit_drone.csv', index=False)

In [20]:
merged_data['기업주소'] = merged_data['기업주소'].apply(lambda x: x.split('지도보기·주소복사')[0].replace('근무지역', '').strip())

# 결과 확인 및 저장
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,니어스랩,경력경력 2~10년,회사내규에 따름,"서울 강남구 논현로 417, 4,5층 4,5층",드론,• 드론으로 획득한 데이터를 관리하고 가공하는 B2B 웹 서비스 Front-end 개발,[JavaScriptTypeScriptReactReact Hook FormNext....
1,니어스랩,경력경력 3~10년,회사내규에 따름,"서울 강남구 논현로 417, 4,5층 4,5층",드론,• Arm 기반 Embedded Linux 개발\n- 시스템 구성에 필요한 부품 및...,"[C++ARMROSFWMCUEmbedded LinuxEmbeddedC, • Came..."
2,에너자이,경력신입~10년,회사내규에 따름,서울 강남구 테헤란로27길 27,드론,• Computer Vision 알고리즘 개발\n - 다양한 형태의 데이터 처리 ...,[AI/인공지능MachineLearningDeepLearningTensorFlowP...
3,에너자이,경력신입~10년,회사내규에 따름,서울 강남구 테헤란로27길 27,드론,• Edge device targeted AI compiler 개발\n• 경량화 파...,[C++HaskellDeepLearningF#TensorFlowScalaPyTorc...
4,에너자이,경력경력 2~3년,회사내규에 따름,서울 강남구 테헤란로27길 27,드론,• CCTV 카메라 파이프라인상에서 에너자이의 AI 모델이 원활하게 작동하도록 해요...,"[AI/인공지능C++Embedded LinuxEmbeddedC, • 임베디드, 모바..."


In [21]:
merged_data['고용형태'] = merged_data['고용형태'].str.replace('경력경력', '경력')

In [22]:
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,니어스랩,경력 2~10년,회사내규에 따름,"서울 강남구 논현로 417, 4,5층 4,5층",드론,• 드론으로 획득한 데이터를 관리하고 가공하는 B2B 웹 서비스 Front-end 개발,[JavaScriptTypeScriptReactReact Hook FormNext....
1,니어스랩,경력 3~10년,회사내규에 따름,"서울 강남구 논현로 417, 4,5층 4,5층",드론,• Arm 기반 Embedded Linux 개발\n- 시스템 구성에 필요한 부품 및...,"[C++ARMROSFWMCUEmbedded LinuxEmbeddedC, • Came..."
2,에너자이,경력신입~10년,회사내규에 따름,서울 강남구 테헤란로27길 27,드론,• Computer Vision 알고리즘 개발\n - 다양한 형태의 데이터 처리 ...,[AI/인공지능MachineLearningDeepLearningTensorFlowP...
3,에너자이,경력신입~10년,회사내규에 따름,서울 강남구 테헤란로27길 27,드론,• Edge device targeted AI compiler 개발\n• 경량화 파...,[C++HaskellDeepLearningF#TensorFlowScalaPyTorc...
4,에너자이,경력 2~3년,회사내규에 따름,서울 강남구 테헤란로27길 27,드론,• CCTV 카메라 파이프라인상에서 에너자이의 AI 모델이 원활하게 작동하도록 해요...,"[AI/인공지능C++Embedded LinuxEmbeddedC, • 임베디드, 모바..."


In [40]:
merged_data.to_csv('../data/project_eda_dataset/jumpit_drone.csv', index=False)